<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

Importing the necessary libraries

In [1]:
# importing pandas and Numpy Libraries
import numpy as np
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup

### Download and Explore Dataset

In [2]:
# get the response in the form of html from wikipedia page
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
Postal_codes=soup.find('table',{'class':"wikitable"})

In [4]:
Postal_codes

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

##### Loading the table into dataframe

In [5]:
df=pd.read_html(str(Postal_codes))
# convert list to dataframe
df=pd.DataFrame(df[0])
print(df.head())

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


Printing the size of the dataframe

In [6]:
df.shape

(180, 3)

In [7]:
# Exclude the rows with borough value = 'Not assigned'
df1 = df[df.Borough != 'Not assigned']
df1

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [8]:
#If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
df1.loc[df1.Neighbourhood =='Not assigned','Neighbourhood'] = df1['Borough']
df1.columns = df1.columns.str.replace(' ', '')
df1.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
#Printing the size of the dataframe
df1.shape

(103, 3)

##### Loading the csv file that has the geographical coordinates of each postal code from http://cocl.us/Geospatial_data

In [10]:
# Loading the csv file that has the geographical coordinates of each postal code from http://cocl.us/Geospatial_data
toronto_data = pd.read_csv("http://cocl.us/Geospatial_data")
# Removing the space from column name
toronto_data.columns = toronto_data.columns.str.replace(' ', '')
toronto_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the geographical coordinates data from the new dataframe to our toronto_data

In [11]:
toronto_postal = df1.merge(toronto_data[["PostalCode","Latitude","Longitude"]], on="PostalCode", how="left")
toronto_postal["Latitude"] = toronto_postal["Latitude"].fillna(0)
toronto_postal["Longitude"] = toronto_postal["Longitude"].fillna(0)
toronto_postal.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'RBB2FCEJEEFCWV12JE1FMH1TSO00B0XZZK4E5V4HOD4INFJA' # your Foursquare ID
CLIENT_SECRET = 'BBAZCWCW3MV5UZMPR5PFFJEZ0L11LSIUC5EZ0CLWSL1BCSU1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [13]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 94 kB 6.1 MB/s  eta 0:00:01
Libraries imported.


In [14]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

As we did with all of New York City, let's visualize the postal codes in Toronto.

In [16]:

# add markers to map
for lat, lng, borough,postalcode in zip(toronto_postal['Latitude'], toronto_postal['Longitude'], toronto_postal['Borough'],toronto_postal['PostalCode']):
    label = '{}, {}'.format(borough,postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Exploring Neighborhoods in Toronto


In [17]:
CLIENT_ID = 'RBB2FCEJEEFCWV12JE1FMH1TSO00B0XZZK4E5V4HOD4INFJA' # your Foursquare ID
CLIENT_SECRET = 'BBAZCWCW3MV5UZMPR5PFFJEZ0L11LSIUC5EZ0CLWSL1BCSU1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RBB2FCEJEEFCWV12JE1FMH1TSO00B0XZZK4E5V4HOD4INFJA
CLIENT_SECRET:BBAZCWCW3MV5UZMPR5PFFJEZ0L11LSIUC5EZ0CLWSL1BCSU1


In [18]:
toronto_postal.loc[0, 'Borough']

'North York'

In [19]:
neighborhood_latitude = toronto_postal.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_postal.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_postal.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of North York are 43.7532586, -79.3296565.


#### Let's create a function to repeat the process to get the venues returned by Foursquare to all the neighborhoods in Toronto

In [20]:


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



##### Now write the code to run the above function on each Borough and create a new dataframe called toronto_venues.

In [21]:
toronto_venues = getNearbyVenues(names=toronto_postal['Borough'],
                                   latitudes=toronto_postal['Latitude'],
                                   longitudes=toronto_postal['Longitude']
                                  )
#### The size of the resulting dataframe
print(toronto_venues.shape)

North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

In [22]:
toronto_venues.head()

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,North York,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


## Analyze Each Neighborhood


In [23]:
toronto_venues.groupby('Borough').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,108,108,108,108,108,108
Downtown Toronto,1212,1212,1212,1212,1212,1212
East Toronto,117,117,117,117,117,117
East York,75,75,75,75,75,75
Etobicoke,74,74,74,74,74,74
Mississauga,14,14,14,14,14,14
North York,247,247,247,247,247,247
Scarborough,87,87,87,87,87,87
West Toronto,154,154,154,154,154,154


### Analyze Each Neighborhood


In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.009259,...,0.000000,0.000000,0.000000,0.009259,0.000000,0.000000,0.000000,0.000000,0.000000,0.009259
1,Downtown Toronto,0.000000,0.000825,0.000825,0.000825,0.000825,0.00165,0.002475,0.000825,0.014026,...,0.011551,0.001650,0.000000,0.003300,0.000000,0.006601,0.000825,0.000000,0.000000,0.005776
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.025641,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017094
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.013333,0.000000,0.013333,0.000000,0.000000,0.000000,0.000000,0.013333
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.013514,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004049,0.000000,0.004049,0.000000,0.000000,0.00000,0.000000,0.000000,0.008097,...,0.000000,0.004049,0.000000,0.008097,0.000000,0.000000,0.000000,0.000000,0.004049,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.011494,...,0.000000,0.000000,0.000000,0.022989,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.019481,0.000000,0.000000,0.006494,0.000000,0.006494,0.000000,0.000000,0.000000,0.012987
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000


In [26]:
toronto_grouped.shape

(10, 272)

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.06
2     Pizza Place  0.06
3  Sandwich Place  0.06
4            Park  0.06


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.05
2           Restaurant  0.03
3  Japanese Restaurant  0.03
4                Hotel  0.03


----East Toronto----
                venue  freq
0         Coffee Shop  0.07
1    Greek Restaurant  0.06
2  Italian Restaurant  0.04
3             Brewery  0.04
4         Pizza Place  0.03


----East York----
                 venue  freq
0          Coffee Shop  0.07
1                 Park  0.05
2                 Bank  0.05
3  Sporting Goods Shop  0.04
4         Burger Joint  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1     Coffee Shop  0.05
2        Pharmacy  0.05
3  Sandwich Place  0.05
4             Gym  0.04


----Mississauga----
                       venue  freq
0                Coffee S

function to sort the venues in descending order.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Clothing Store,Dessert Shop,Restaurant,Health & Beauty Service
1,Downtown Toronto,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
2,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Brewery,Ice Cream Shop,Restaurant,Fast Food Restaurant,Park,Bakery,Pizza Place
3,East York,Coffee Shop,Park,Bank,Sporting Goods Shop,Pizza Place,Sandwich Place,Burger Joint,Skating Rink,Intersection,Beer Store
4,Etobicoke,Pizza Place,Pharmacy,Sandwich Place,Coffee Shop,Fast Food Restaurant,Grocery Store,Gym,Bakery,Liquor Store,Convenience Store


###  Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 5 clusters.

In [63]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 4, 1, 0, 2, 0, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [64]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [65]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Clothing Store,Dessert Shop,Restaurant,Health & Beauty Service
1,0,Downtown Toronto,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
2,0,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Brewery,Ice Cream Shop,Restaurant,Fast Food Restaurant,Park,Bakery,Pizza Place
3,2,East York,Coffee Shop,Park,Bank,Sporting Goods Shop,Pizza Place,Sandwich Place,Burger Joint,Skating Rink,Intersection,Beer Store
4,4,Etobicoke,Pizza Place,Pharmacy,Sandwich Place,Coffee Shop,Fast Food Restaurant,Grocery Store,Gym,Bakery,Liquor Store,Convenience Store


In [66]:
toronto_final_merged = pd.merge(toronto_postal,
                 neighborhoods_venues_sorted,
                 on='Borough', 
                 how='left')


toronto_final_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Bank,Pizza Place,Fast Food Restaurant,Grocery Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Bank,Pizza Place,Fast Food Restaurant,Grocery Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Bank,Pizza Place,Fast Food Restaurant,Grocery Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant


visualize the resulting clusters

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon,pos, cluster in zip(toronto_final_merged['Latitude'], toronto_final_merged['Longitude'],toronto_final_merged['PostalCode'], toronto_final_merged['Cluster Labels']):
    label = folium.Popup(str(pos) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [68]:
toronto_final_merged.loc[toronto_final_merged['Cluster Labels'] == 0, toronto_final_merged.columns[[1] + list(range(5, toronto_final_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Bank,Pizza Place,Fast Food Restaurant,Grocery Store
1,North York,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Bank,Pizza Place,Fast Food Restaurant,Grocery Store
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
3,North York,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Bank,Pizza Place,Fast Food Restaurant,Grocery Store
4,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
92,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
96,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
97,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant
99,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Seafood Restaurant


#### Cluster 2

In [69]:
toronto_final_merged.loc[toronto_final_merged['Cluster Labels'] == 1, toronto_final_merged.columns[[1] + list(range(5, toronto_final_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,Mississauga,1,Coffee Shop,Hotel,Burrito Place,American Restaurant,Food Truck,Sandwich Place,Gas Station,Intersection,Fried Chicken Joint,Middle Eastern Restaurant
